In [70]:
import os
from PIL import Image
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import imageio
import cv2
import glob
from matplotlib.widgets import RectangleSelector
from matplotlib.patches import Rectangle
import pandas as pd

In [72]:
images_path = r'filtrirane_slike'
roi1_folder = r'templates' #-----------------roi1-----------------
roi2_folder = r'medijastinum' #--------------roi2------------------

In [74]:
#---------------------odabrano filtriranje----------------------
def filtriranje(images, method='non_local_means'):
    denoised_images = []
    for img in images:
        denoised_img_nlm = cv2.fastNlMeansDenoising(img, None, h=10, templateWindowSize=7, searchWindowSize=21)
        denoised_img_nlm_bilateral = cv2.bilateralFilter(denoised_img_nlm, d=30, sigmaColor=30, sigmaSpace=30)
        denoised_images.append(denoised_img_nlm_bilateral)
    return denoised_images

In [76]:
# --------------------- ucitavanje slika i sablona ---------------------
def ucitaj_slike(folder, target_size=(512, 512)):
    slike = []
    files = [f for f in sorted(os.listdir(folder)) if f.endswith(('png', 'jpg', 'jpeg'))]
    
    for f in files:
        img_path = os.path.join(folder, f)
        img = np.array(Image.open(img_path).convert('L'))
        slike.append(img)
    return slike

def ucitaj_sablone(folder):
    sabloni = []
    for f in sorted(os.listdir(folder)):
        if f.endswith(('png', 'jpg', 'jpeg')):
            img_path = os.path.join(folder, f)
            img = np.array(Image.open(img_path).convert('L'))
            sabloni.append(img)  # zadrzava originalnu velič=cinu
    return sabloni

In [78]:
slike_filt = ucitaj_slike(images_path) 
roi1 = ucitaj_sablone(roi1_folder)
roi2 = ucitaj_sablone(roi2_folder)
roi1_filt = filtriranje(roi1)
roi2_filt = filtriranje(roi2)

In [80]:
# -------------------------------------prvi pristup-------------------------------
# --------------------- template matching ---------------------
def pronadji_slican_region(img, sablon):
    h_sablona, w_sablona = sablon.shape[:2]
    h_slike, w_slike = img.shape[:2]

    # ako je sablon veci od slike -> smanjiti
    if h_slike < h_sablona or w_slike < w_sablona:
        scale_factor = min(h_slike / h_sablona, w_slike / w_sablona)
        sablon = cv2.resize(sablon, (int(w_sablona * scale_factor), int(h_sablona * scale_factor)))

    result = cv2.matchTemplate(img, sablon, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    start_x, start_y = max_loc
    h, w = sablon.shape[:2]
    return (start_x, start_y, w, h), max_val

#------------------------izdvajanje najboljih roi------------------------

def najbolji_roi(img, sabloni):
    best_roi = None
    best_score = -np.inf
    for sablon in sabloni:
        roi, score = pronadji_slican_region(img, sablon)
        if score > best_score:
            best_score = score
            best_roi = roi
    return best_roi, best_score

# --------------------- crtanje ROI ---------------------
def prikazi_roi(img, roi1, roi2):
    img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    if roi1:
        x, y, w, h = roi1
        cv2.rectangle(img_color, (x, y), (x+w, y+h), (0, 0, 255), 2)  # crveni
    if roi2:
        x, y, w, h = roi2
        cv2.rectangle(img_color, (x, y), (x+w, y+h), (0, 255, 0), 2)  # zeleni
    return img_color

In [82]:
#--------------------------poboljsani pristup------------------------------------
# --------------------- multivelicinska template matching ---------------------
#--------------------------Gausova piramida-----------------------------
def im_pyr_decomp(im, N):
    GPyr = []
    for i in range(N):
        GPyr.append(im) # originalna slika
        g = cv2.pyrDown(im, borderType=cv2.BORDER_REPLICATE)
        im = g 
    Res = im
    return GPyr, Res
    
def najbolji_roi_multiscale(img, sabloni, max_levels=3):
    """
    img: filtrirana slika
    sabloni: lista filtriranih sablona (bez promene dimenzije)
    max_levels: koliko nivoa piramide da se koristi
    """
    best_roi = None
    best_score = -np.inf
    
    GPyr, _ = im_pyr_decomp(img, max_levels) 
    
    for sablon in sabloni:
        for level, img_lvl in enumerate(GPyr):
            roi_lvl, score = pronadji_slican_region(img_lvl, sablon)
            
            if score > best_score:
                best_score = score
                scale_factor = 2 ** level
                x, y, w, h = roi_lvl
                # skaliramo koordinate dok je velicina roi-a ista
                best_roi = (x * scale_factor, y * scale_factor, w, h)
    
    return best_roi, best_score

In [84]:
image_names = []

for filepath in glob.glob(os.path.join(images_path, "*.png")):  
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE) 
    if img is not None:
        image_names.append(os.path.basename(filepath)) 

In [86]:
def sacuvaj_koef_korelacije():
    sabloni1 = roi1_filt
    sabloni2 = roi2_filt
    if not sabloni1 or not sabloni2:
        print("Nema učitanih šablona.")
        return
    
    rezultati_prvi = []
    rezultati_drugi = []

    for i, image_filt in enumerate(slike_filt):
        ime_slike = image_names[i] if i < len(image_names) else f"slika_{i:04d}"

        # ---------- PRVI PRISTUP ----------
        roi1_prvi, score1_prvi = najbolji_roi(image_filt, sabloni1)
        roi2_prvi, score2_prvi = najbolji_roi(image_filt, sabloni2)

        kandidati_prvi = []
        if score1_prvi > 0.9:
            kandidati_prvi.append(("roi1", score1_prvi))
        if score2_prvi > 0.9:
            kandidati_prvi.append(("roi2", score2_prvi))

        if kandidati_prvi:  # uzmi samo maksimum
            grupa, score = max(kandidati_prvi, key=lambda x: x[1])
            rezultati_prvi.append({
                "slika": ime_slike,
                "grupa": grupa,
                "score": score
            })

        # ---------- DRUGI PRISTUP (multiscale) ----------
        roi1_drugi, score1_drugi = najbolji_roi_multiscale(image_filt, sabloni1)
        roi2_drugi, score2_drugi = najbolji_roi_multiscale(image_filt, sabloni2)

        kandidati_drugi = []
        if score1_drugi > 0.9:
            kandidati_drugi.append(("roi1", score1_drugi))
        if score2_drugi > 0.9:
            kandidati_drugi.append(("roi2", score2_drugi))

        if kandidati_drugi:  # uzmi samo maksimum
            grupa, score = max(kandidati_drugi, key=lambda x: x[1])
            rezultati_drugi.append({
                "slika": ime_slike,
                "grupa": grupa,
                "score": score
            })

    # ----------cuvanje u folder ----------
    folder = "koeficijenti_korelacije"
    os.makedirs(folder, exist_ok=True)

    output_path_1 = os.path.join(folder, "rezultati_prvi_pristup.xlsx")
    output_path_2 = os.path.join(folder, "rezultati_drugi_multiscale.xlsx")

    df_prvi = pd.DataFrame(rezultati_prvi)
    df_drugi = pd.DataFrame(rezultati_drugi)

    df_prvi.to_excel(output_path_1, index=False, engine="openpyxl")
    df_drugi.to_excel(output_path_2, index=False, engine="openpyxl")

    print(f"Tabela prvog pristupa sačuvana u: {output_path_1}")
    print(f"Tabela drugog pristupa sačuvana u: {output_path_2}")

In [88]:
sacuvaj_koef_korelacije()

Tabela prvog pristupa sačuvana u: koeficijenti_korelacije\rezultati_prvi_pristup.xlsx
Tabela drugog pristupa sačuvana u: koeficijenti_korelacije\rezultati_drugi_multiscale.xlsx


In [90]:
def sacuvaj_sve_koeficijente():
    sabloni1 = roi1_filt
    sabloni2 = roi2_filt
    if not sabloni1 or not sabloni2:
        print("Nema učitanih šablona.")
        return

    rezultati_prvi = []
    rezultati_drugi = []

    for i, image_filt in enumerate(slike_filt):
        ime_slike = image_names[i] if i < len(image_names) else f"slika_{i:04d}"

        # ---------- PRVI PRISTUP ----------
        roi1_prvi, score1_prvi = najbolji_roi(image_filt, sabloni1)
        roi2_prvi, score2_prvi = najbolji_roi(image_filt, sabloni2)

        if roi1_prvi is not None:
            x, y, w, h = roi1_prvi
            rezultati_prvi.append({
                "Ime slike": ime_slike,
                "Koordinata x": x,
                "Koordinata y": y,
                "Širina": w,
                "Visina": h,
                "Tip template-a": "roi1",
                "Korelacioni koeficijent": score1_prvi
            })
        if roi2_prvi is not None:
            x, y, w, h = roi2_prvi
            rezultati_prvi.append({
                "Ime slike": ime_slike,
                "Koordinata x": x,
                "Koordinata y": y,
                "Širina": w,
                "Visina": h,
                "Tip template-a": "roi2",
                "Korelacioni koeficijent": score2_prvi
            })

        # ---------- DRUGI PRISTUP (multiscale) ----------
        roi1_drugi, score1_drugi = najbolji_roi_multiscale(image_filt, sabloni1)
        roi2_drugi, score2_drugi = najbolji_roi_multiscale(image_filt, sabloni2)

        if roi1_drugi is not None:
            x, y, w, h = roi1_drugi
            rezultati_drugi.append({
                "Ime slike": ime_slike,
                "Koordinata x": x,
                "Koordinata y": y,
                "Širina": w,
                "Visina": h,
                "Tip template-a": "roi1",
                "Korelacioni koeficijent": score1_drugi
            })
        if roi2_drugi is not None:
            x, y, w, h = roi2_drugi
            rezultati_drugi.append({
                "Ime slike": ime_slike,
                "Koordinata x": x,
                "Koordinata y": y,
                "Širina": w,
                "Visina": h,
                "Tip template-a": "roi2",
                "Korelacioni koeficijent": score2_drugi
            })

    # ---------- ČUVANJE U FOLDER ----------
    folder = "koeficijenti_korelacije"
    os.makedirs(folder, exist_ok=True)

    output_path_1 = os.path.join(folder, "detaljni_rezultati_prvi.xlsx")
    output_path_2 = os.path.join(folder, "detaljni_rezultati_drugi_multiscale.xlsx")

    df_prvi = pd.DataFrame(rezultati_prvi)
    df_drugi = pd.DataFrame(rezultati_drugi)

    df_prvi.to_excel(output_path_1, index=False, engine="openpyxl")
    df_drugi.to_excel(output_path_2, index=False, engine="openpyxl")

    print(f"Detaljna tabela prvog pristupa sačuvana u: {output_path_1}")
    print(f"Detaljna tabela drugog pristupa sačuvana u: {output_path_2}")

In [92]:
sacuvaj_sve_koeficijente()

Detaljna tabela prvog pristupa sačuvana u: koeficijenti_korelacije\detaljni_rezultati_prvi.xlsx
Detaljna tabela drugog pristupa sačuvana u: koeficijenti_korelacije\detaljni_rezultati_drugi_multiscale.xlsx


In [94]:
#-----------------------preklapanje roi-a preko 90 - cuvamo u posebnu tabelu---------------------------------
#-----------------------iz preklopljenih roi gledamo roi sa vecim koef korelacije------------------------------
# Funkcija za izracunavanje procenta piksela srca unutar šupljine
def calculate_heart_within_hollow_percentage(heart_roi, hollow_roi, img_shape):
    if heart_roi is None or hollow_roi is None:
        return 0.0

    # Izračunavanje koordinata ROI-ova
    x1, y1, w1, h1 = heart_roi
    x2, y2, w2, h2 = hollow_roi

    # Kreiranje maski za srce i šupljinu
    heart_mask = np.zeros(img_shape, dtype=np.uint8)
    hollow_mask = np.zeros(img_shape, dtype=np.uint8)
    cv2.rectangle(heart_mask, (x1, y1), (x1 + w1, y1 + h1), 255, -1)
    cv2.rectangle(hollow_mask, (x2, y2), (x2 + w2, y2 + h2), 255, -1)

    # Izračunavanje presjeka između maski
    intersect_mask = cv2.bitwise_and(heart_mask, hollow_mask)
    heart_area = np.sum(heart_mask > 0) 
    intersect_area = np.sum(intersect_mask > 0)

    if heart_area == 0:
        return 0.0

    # Procenat piksela srca unutar šupljine
    percentage = (intersect_area / heart_area) * 100
    return percentage

In [102]:
# Glavna funkcija za obradu slika, izračunavanje procenta 
def process_images(excel_output_path):
    sabloni1 = roi1_filt # srce
    sabloni2 = roi2_filt # supljine
    
    results = []

    for i, img in enumerate(slike_filt):
        ime_slike = image_names[i] if i < len(image_names) else f"slika_{i:04d}"
        # Pretraga ROI-ja na osnovu šablona
        best_roi1, _ = najbolji_roi_multiscale(img, sabloni1)
        best_roi2, _ = najbolji_roi_multiscale(img, sabloni2)

        # Izračunavanje procenta piksela srca unutar šupljine
        percentage = calculate_heart_within_hollow_percentage(best_roi1, best_roi2, img.shape)

        # Čuvanje rezultata
        results.append({
            "Ime slike": ime_slike,
            "Procenat piksela srca unutar supljine": percentage
        })


    # Čuvanje rezultata u Excel tabelu
    df = pd.DataFrame(results) # Ime slike | Procenat piksela srca unutar supljine
    df.to_excel(excel_output_path, index=False)
    print(f"Rezultati sačuvani u Excel tabelu: {excel_output_path}")


excel_output_path = r'koeficijenti_korelacije/rezultati_procenat_preklapanja_multiscale.xlsx'

# Procesiranje slika 
process_images(excel_output_path)

Rezultati sačuvani u Excel tabelu: koeficijenti_korelacije/rezultati_procenat_preklapanja_multiscale.xlsx
